# Problem 5.3  

**Integrated Energy Grids**

**Problem 5.3**

**Assume we have three buses (Denmark, Sweden, and Norway) with nominal voltage $V_{nom}$= 2000 V connected by three transmission lines. In each of the buses, there is a gas power generator whose variable cost is 50 EUR/MWh and installed capacity is 50 MW. In the Denmark bus, there is a wind generator whose variable cost is zero and whose installed capacity is 200 MW. The transmission lines have a unitary resistance $r$=0.01 and reactance $x$=0.1, and nominal capacity $S_{nom}=100$ VA. The demand is 50 MW for Denmark and Sweden and 30 MW for Norway. Using Python for Power System Analysis (PyPSA):**

**a) Calculate the optimal dispatch that minimizes the total system cost, the energy produced by each generator, and the power flows along the transmission lines using AC power flow representation.**

**b) Calculate the optimal dispatch that minimizes the total system cost,  the energy produced by each generator, and the power flows along the transmission lines using a linearized approximation (DC optimal power flow).**

**c) Calculate the optimal dispatch that minimizes the total system cost,  the energy produced by each generator, and the power flows along the transmission lines using the Net Transfer Capacity (NTC) approach for the transmission lines.**

:::{note}
If you have not yet set up Python on your computer, you can execute this tutorial in your browser via [Google Colab](https://colab.research.google.com/). Click on the rocket in the top right corner and launch "Colab". If that doesn't work download the `.ipynb` file and import it in [Google Colab](https://colab.research.google.com/).

Then install the following packages by executing the following command in a Jupyter cell at the top of the notebook.

```sh
!pip install numpy pypsa
```
:::

In [139]:
import numpy as np
import pypsa

We start by creating the network object and adding the three buses corresponding to Denmark, Sweden, and Norway. 

In [140]:
network = pypsa.Network()

In [141]:
for node in ["Denmark", "Sweden", "Norway"]:
    network.add("Bus", "bus {}".format(node), v_nom=2000)

network.buses

,v_nom,type,x,y,carrier,unit,v_mag_pu_set,v_mag_pu_min,v_mag_pu_max,control,generator,sub_network
Bus,,,,,,,,,,,,
bus Denmark,2000.0,,0.0,0.0,AC,,1.0,0.0,inf,PQ,,
bus Sweden,2000.0,,0.0,0.0,AC,,1.0,0.0,inf,PQ,,
bus Norway,2000.0,,0.0,0.0,AC,,1.0,0.0,inf,PQ,,


We add the three lines connecting the buses

In [142]:
network.add("Line", "line DK-SW", bus0 = "bus Denmark", bus1 = "bus Sweden", s_nom = 100, x=0.1, r=0.01)
network.add("Line", "line DK-NO", bus0 = "bus Denmark", bus1 = "bus Norway", s_nom = 100, x=0.1, r=0.01)
network.add("Line", "line SW-NO", bus0 = "bus Sweden",  bus1 = "bus Norway", s_nom = 100, x=0.1, r=0.01)

network.lines

,bus0,bus1,type,x,r,g,b,s_nom,s_nom_mod,s_nom_extendable,...,v_ang_min,v_ang_max,sub_network,x_pu,r_pu,g_pu,b_pu,x_pu_eff,r_pu_eff,s_nom_opt
Line,,,,,,,,,,,,,,,,,,,,,
line DK-SW,bus Denmark,bus Sweden,,0.1,0.01,0.0,0.0,100.0,0.0,False,...,-inf,inf,,0.0,0.0,0.0,0.0,0.0,0.0,0.0
line DK-NO,bus Denmark,bus Norway,,0.1,0.01,0.0,0.0,100.0,0.0,False,...,-inf,inf,,0.0,0.0,0.0,0.0,0.0,0.0,0.0
line SW-NO,bus Sweden,bus Norway,,0.1,0.01,0.0,0.0,100.0,0.0,False,...,-inf,inf,,0.0,0.0,0.0,0.0,0.0,0.0,0.0


We add the generators

In [143]:
for node in ["Denmark", "Sweden", "Norway"]:
    network.add("Generator", "gas {}".format(node), bus="bus {}".format(node), 
                p_nom=50, 
                marginal_cost=50) #EUR/MWh_elec    
network.add("Generator", "wind Denmark", bus="bus Denmark", 
            p_nom=200, 
            marginal_cost=10)
network.generators

,bus,control,type,p_nom,p_nom_mod,p_nom_extendable,p_nom_min,p_nom_max,p_min_pu,p_max_pu,...,min_up_time,min_down_time,up_time_before,down_time_before,ramp_limit_up,ramp_limit_down,ramp_limit_start_up,ramp_limit_shut_down,weight,p_nom_opt
Generator,,,,,,,,,,,,,,,,,,,,,
gas Denmark,bus Denmark,PQ,,50.0,0.0,False,0.0,inf,0.0,1.0,...,0,0,1,0,NaN,NaN,1.0,1.0,1.0,0.0
gas Sweden,bus Sweden,PQ,,50.0,0.0,False,0.0,inf,0.0,1.0,...,0,0,1,0,NaN,NaN,1.0,1.0,1.0,0.0
gas Norway,bus Norway,PQ,,50.0,0.0,False,0.0,inf,0.0,1.0,...,0,0,1,0,NaN,NaN,1.0,1.0,1.0,0.0
wind Denmark,bus Denmark,PQ,,200.0,0.0,False,0.0,inf,0.0,1.0,...,0,0,1,0,NaN,NaN,1.0,1.0,1.0,0.0


We add the loads.

In [144]:
for node in ["Denmark", "Sweden"]:
    network.add("Load", "load {}".format(node), 
                bus="bus {}".format(node), 
                p_set=50)
network.add("Load", "load Norway", 
                bus="bus Norway", 
                p_set=30)
network.loads

,bus,carrier,type,p_set,q_set,sign,active
Load,,,,,,,
load Denmark,bus Denmark,,,50.0,0.0,-1.0,True
load Sweden,bus Sweden,,,50.0,0.0,-1.0,True
load Norway,bus Norway,,,30.0,0.0,-1.0,True


We optimize searching for the minimum system cost.

In [145]:
network.optimize()
#network.optimize(solver='gurobi', assign_all_duals=True)

Index(['bus Denmark', 'bus Sweden', 'bus Norway'], dtype='object', name='Bus')
Index(['line DK-SW', 'line DK-NO', 'line SW-NO'], dtype='object', name='Line')
Index(['bus Denmark', 'bus Sweden', 'bus Norway'], dtype='object', name='Bus')
Index(['line DK-SW', 'line DK-NO', 'line SW-NO'], dtype='object', name='Line')
INFO:linopy.model: Solve problem using Highs solver
INFO:linopy.io: Writing time: 0.03s
INFO:linopy.constants: Optimization successful: 
Status: ok
Termination condition: optimal
Solution: 7 primals, 18 duals
Objective: 1.30e+03
Solver model: available
Solver message: optimal

INFO:pypsa.optimization.optimize:The shadow-prices of the constraints Generator-fix-p-lower, Generator-fix-p-upper, Line-fix-s-lower, Line-fix-s-upper, Kirchhoff-Voltage-Law were not assigned to the network.


('ok', 'optimal')

Now, we can look at what is the optimal dispatch form every genenerator. As expected, the wind generator is producing power to supply the demand in every node.

In [146]:
network.generators_t.p

Generator,gas Denmark,gas Sweden,gas Norway,wind Denmark
snapshot,,,,
now,-0.0,-0.0,-0.0,130.0


We can se the optimal dispatch in the generators and then solve the non-linear power flow using a Newton-Raphson method.

In [147]:
network.generators_t.p_set = network.generators_t.p
network.pf()

INFO:pypsa.pf:Performing non-linear load-flow on AC sub-network <pypsa.components.SubNetwork object at 0x000001E78999A720> for snapshots Index(['now'], dtype='object', name='snapshot')


{'n_iter': SubNetwork  0
 snapshot     
 now         2,
 'error': SubNetwork             0
 snapshot                
 now         7.362638e-09,
 'converged': SubNetwork     0
 snapshot        
 now         True}

ok, the solution converge, we can check now the active power flow on the lines.

In [148]:
network.lines_t.p0

,line DK-SW,line DK-NO,line SW-NO
snapshot,,,
now,43.333338,36.66667,-6.666667


We can also check the voltage angles on the buses

In [149]:
network.buses_t.v_ang * 180 / np.pi

Bus,bus Denmark,bus Sweden,bus Norway
snapshot,,,
now,0.0,-0.000062,-0.000053


and their mangitudes

In [150]:
network.buses_t.v_mag_pu

Bus,bus Denmark,bus Sweden,bus Norway
snapshot,,,
now,1.0,1.0,1.0


**b) Calculate the optimal dispatch that minimizes the total system cost,  the energy produced by each generator, and the power flows along the transmission lines using a linearized approximation (DC optimal power flow)**

In this case, since the voltage angles are very small, the linear power flow should be a good approximation. We can calculate the power flows in the line using the linear power flow (lpf) and check that we obtained a very similar result.

In [151]:
network.lpf()

INFO:pypsa.pf:Performing linear load-flow on AC sub-network <pypsa.components.SubNetwork object at 0x000001E78999A720> for snapshot(s) Index(['now'], dtype='object', name='snapshot')


In [152]:
network.lines_t.p0

,line DK-SW,line DK-NO,line SW-NO
snapshot,,,
now,43.333333,36.666667,-6.666667


**c) Calculate the optimal dispatch that minimizes the total system cost,  the energy produced by each generator, and the power flows along the transmission lines using Net Transfer Capacity (NTC) approach for the transmission lines.**

We can create the problem again and this time use links to represent lines using only their Net Transfer Capacities

In [155]:
network = pypsa.Network()
for node in ["Denmark", "Sweden", "Norway"]:
    network.add("Bus", "bus {}".format(node), v_nom=2000)

network.add("Link","line DK-SW", bus0 = "bus Denmark", bus1 = "bus Sweden", p_nom = 100, p_min_pu=-1)
network.add("Link","line DK-NO", bus0 = "bus Denmark", bus1 = "bus Norway", p_nom =100, p_min_pu=-1)
network.add("Link","line SW-NO", bus0 = "bus Sweden", bus1 = "bus Norway", p_nom = 100, p_min_pu=-1)

for node in ["Denmark", "Sweden"]:
    network.add("Load", "load {}".format(node), 
                bus="bus {}".format(node), 
                p_set=50)
network.add("Load", "load Norway", 
                bus="bus Norway", 
                p_set=30)
network.loads

for node in ["Denmark", "Sweden", "Norway"]:
    network.add("Generator", "gas {}".format(node), bus="bus {}".format(node), 
                p_nom=50, 
                marginal_cost=50) #EUR/MWh_elec    
network.add("Generator", "wind Denmark", bus="bus Denmark", 
            p_nom=200, 
            marginal_cost=10)


network.optimize()

Index(['bus Denmark', 'bus Sweden', 'bus Norway'], dtype='object', name='Bus')
Index(['line DK-SW', 'line DK-NO', 'line SW-NO'], dtype='object', name='Link')
Index(['bus Denmark', 'bus Sweden', 'bus Norway'], dtype='object', name='Bus')
Index(['line DK-SW', 'line DK-NO', 'line SW-NO'], dtype='object', name='Link')
INFO:linopy.model: Solve problem using Highs solver
INFO:linopy.io: Writing time: 0.04s
INFO:linopy.constants: Optimization successful: 
Status: ok
Termination condition: optimal
Solution: 7 primals, 17 duals
Objective: 1.30e+03
Solver model: available
Solver message: optimal

INFO:pypsa.optimization.optimize:The shadow-prices of the constraints Generator-fix-p-lower, Generator-fix-p-upper, Link-fix-p-lower, Link-fix-p-upper were not assigned to the network.


('ok', 'optimal')

In [156]:
network.generators_t.p

Generator,gas Denmark,gas Sweden,gas Norway,wind Denmark
snapshot,,,,
now,-0.0,-0.0,-0.0,130.0


In [157]:
network.links_t.p0

Link,line DK-SW,line DK-NO,line SW-NO
snapshot,,,
now,100.0,-20.0,50.0


In this case, the power flows are also compatible with the nodal balances, but they are different from those obtained using pf or lpf.